In [1]:
import pickle
import gc
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn import metrics

In [2]:
train_f = pd.read_pickle('../dataset/train/train_all_feature.pkl')
train_f.shape

(879539, 486)

In [3]:
test_f = pd.read_pickle('../dataset/train/test_all_feature.pkl')
test_f.shape

(215287, 486)

In [4]:
model_path = '../dataset/lightgbmmodel.pkl'

In [5]:
luck = 123456

In [6]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [7]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
#     'max_depth': [2, 6, 12],
    'max_depth': [12],
    'num_leaves': range(30, 50, 4), 

#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [8]:
use_columns = [
    x for x in train_f.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel', 'index'
    ]
]

In [9]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [10]:
clf_feture = train_f.sample(100000, random_state=luck)

In [11]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 5 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   24.0s finished


CPU times: user 16.4 s, sys: 386 ms, total: 16.8 s
Wall time: 29.2 s


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'num_leaves': range(30, 50, 4), 'max_depth': [12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [12]:
clf.best_score_

0.97419999999999995

In [13]:
clf.best_params_

{'max_depth': 12, 'num_leaves': 46}

In [14]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8

clf.best_params_['colsample_bylevel'] = 0.7

# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 56


# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1
clf.best_params_['lambda_l2'] = 10
clf.best_params_['tree_method'] = 'exact'
clf.best_params_['metric'] = 'binary_logloss'



In [36]:
params = {
    'boosting_type': 'gbdt',
#     'objective': 'binary',
    'metric': 'binary_logloss',
    'min_child_weight': 1.5,
#     'num_leaves': 2 ** 7,
    'max_depth':12,
  'num_leaves':80,
#     'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
#     'colsample_bylevel': 0.7,
    'learning_rate': 0.03,
#     'tree_method': 'exact',
}

In [37]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1300,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, learning_rate=0.03,
        max_bin=255, max_depth=12, metric='binary_logloss',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1300, nthread=4, num_leaves=80, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [34]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [38]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel']),
             (test_f[use_columns],
    test_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=50,
)

[1]	valid_0's auc: 0.888304	valid_1's auc: 0.886867
Train until valid scores didn't improve in 50 rounds.
[2]	valid_0's auc: 0.900491	valid_1's auc: 0.89877
[3]	valid_0's auc: 0.901355	valid_1's auc: 0.899617
[4]	valid_0's auc: 0.903032	valid_1's auc: 0.901434
[5]	valid_0's auc: 0.904652	valid_1's auc: 0.902775
[6]	valid_0's auc: 0.90662	valid_1's auc: 0.904734
[7]	valid_0's auc: 0.906916	valid_1's auc: 0.905357
[8]	valid_0's auc: 0.906941	valid_1's auc: 0.905367
[9]	valid_0's auc: 0.907741	valid_1's auc: 0.906048
[10]	valid_0's auc: 0.908214	valid_1's auc: 0.906381
[11]	valid_0's auc: 0.910385	valid_1's auc: 0.908492
[12]	valid_0's auc: 0.910656	valid_1's auc: 0.908622
[13]	valid_0's auc: 0.911095	valid_1's auc: 0.909107
[14]	valid_0's auc: 0.911823	valid_1's auc: 0.910037
[15]	valid_0's auc: 0.911989	valid_1's auc: 0.910078
[16]	valid_0's auc: 0.912341	valid_1's auc: 0.910436
[17]	valid_0's auc: 0.912305	valid_1's auc: 0.910478
[18]	valid_0's auc: 0.912352	valid_1's auc: 0.910554
[19

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, learning_rate=0.03,
        max_bin=255, max_depth=12, metric='binary_logloss',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1300, nthread=4, num_leaves=80, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [39]:
model.best_iteration

995

In [40]:
def get_score(model, train_f):
    result = model.predict_proba(train_f[use_columns])[:, 1]
    train_f['prob'] = result
    train_x = train_f[['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [41]:
train_score = get_score(model, train_f)

0.606518445727


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
test_score = get_score(model, test_f)

0.436158006884


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
acu = model.score(train_f[use_columns], train_f['orderlabel'])
acu

0.97962114243939158

In [44]:
# y_predict = model.predict_proba(train_features[use_columns])[:, 1]
# fpr, tpr, thresholds  = metrics.roc_curve(train_features['orderlabel'], y_predict, pos_label=1)
# auc_score = metrics.auc(fpr, tpr)
# auc_score

In [45]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/06-1542-importance.txt'

In [46]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [47]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'subsample: 0.8,tree_method: exact,num_leaves: 56,learning_rate: 0.1,lambda_l2: 10,colsample_bylevel: 0.7,metric: binary_logloss,colsample_bytree: 0.8,max_depth: 6'

In [48]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in params.items()])
parms

'subsample: 0.7,max_depth: 12,learning_rate: 0.03,boosting_type: gbdt,metric: binary_logloss,colsample_bytree: 0.7,num_leaves: 80,min_child_weight: 1.5'

In [49]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ')

06-15-42:  online  ??  test_score  0.436158006884  train_score  0.606518445727  model_train  0.982162038882  model_test  0.933423731475  accu  0.979621142439  n  995  params  subsample: 0.7,max_depth: 12,learning_rate: 0.03,boosting_type: gbdt,metric: binary_logloss,colsample_bytree: 0.7,num_leaves: 80,min_child_weight: 1.5  shape  (879539, 487)


In [50]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ', file=open('lgb_result.txt', 'a+'))

In [51]:
del train_f, test_f

In [52]:
gc.collect()

72

In [53]:
train_f = pd.read_pickle('../dataset/train/all_feature.pkl')
train_f.shape

(1094826, 486)

In [54]:
n_est = model.best_iteration + 200 if model.best_iteration > 0 else 1500

In [55]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=n_est,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, learning_rate=0.03,
        max_bin=255, max_depth=12, metric='binary_logloss',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1195, nthread=4, num_leaves=80, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [56]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [57]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
)

[1]	valid_0's auc: 0.891907
[2]	valid_0's auc: 0.902652
[3]	valid_0's auc: 0.905077
[4]	valid_0's auc: 0.906258
[5]	valid_0's auc: 0.907583
[6]	valid_0's auc: 0.907787
[7]	valid_0's auc: 0.907909
[8]	valid_0's auc: 0.907854
[9]	valid_0's auc: 0.908379
[10]	valid_0's auc: 0.909376
[11]	valid_0's auc: 0.911621
[12]	valid_0's auc: 0.911785
[13]	valid_0's auc: 0.911998
[14]	valid_0's auc: 0.912642
[15]	valid_0's auc: 0.912686
[16]	valid_0's auc: 0.912654
[17]	valid_0's auc: 0.912553
[18]	valid_0's auc: 0.912656
[19]	valid_0's auc: 0.912595
[20]	valid_0's auc: 0.912555
[21]	valid_0's auc: 0.91241
[22]	valid_0's auc: 0.912475
[23]	valid_0's auc: 0.912824
[24]	valid_0's auc: 0.913099
[25]	valid_0's auc: 0.913067
[26]	valid_0's auc: 0.913119
[27]	valid_0's auc: 0.913127
[28]	valid_0's auc: 0.913108
[29]	valid_0's auc: 0.913683
[30]	valid_0's auc: 0.913918
[31]	valid_0's auc: 0.914045
[32]	valid_0's auc: 0.914265
[33]	valid_0's auc: 0.914326
[34]	valid_0's auc: 0.91475
[35]	valid_0's auc: 0.914

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, learning_rate=0.03,
        max_bin=255, max_depth=12, metric='binary_logloss',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1195, nthread=4, num_leaves=80, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [58]:
pickle.dump(model, open(model_path, 'wb'))

In [59]:
# clf1 =  LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=42, max_depth=6)
# clf2 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=32, max_depth=5)
# clf3 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=27, max_depth=3)
# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
#                           meta_classifier=lr)

# model = sclf